In [2]:
import torch

requires_grad = True sets up the computational tracking on the tensor

In [3]:
x = torch.tensor(2.0, requires_grad=True)

Example function 1: $y(x) = 2 x^{4} + x^{3} + 3 x^{2} + 5 x + 1$

In [6]:
y = 2*x**4 + x**3 + 3*x**2 + 5*x + 1
print(y)

tensor(63., grad_fn=<AddBackward0>)


'AddBackward0'

The backward method of the tensor `object y` set by default the derivative of the y function with respect to x evaluated in 2.0 in the property .grad of the `x object`
$$\frac{dy}{dx}=8(x)^3+3(x)^2+6(x)+5 $$
$$\left. \frac{dy}{dx} \right|_{x=2}=8(2)^3+3(2)^2+6(2)+5 = 64+12+12+5 = 93$$

In [7]:
y.backward()
print(x.grad)

tensor(93.)


### Multistep backpropagation

In [8]:
x = torch.tensor([[1.,2,3],[3 ,2 ,1]], requires_grad=True)
print(x)

tensor([[1., 2., 3.],
        [3., 2., 1.]], requires_grad=True)
